## MAIN SCRIPT

This is old scrpit for some remaping. However, it still can be used for statistics which is located at the end of the file.

In [1]:
import os
import pandas as pd
import shutil

%load_ext autoreload
%autoreload 2

## Generate data - merge

In [2]:
_path = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\CheckedOutNNMD"
_path_true = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\OriginalData"
_output_dir = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256"

# List to store the paths of all .xlsx files
_xlsx_files = []
_xlsx_true_files = []

# Walk through the directory and its subdirectories
for _root, _dirs, _files in os.walk(_path):
    for _file in _files:
        if _file.endswith('.xlsx'):
            _xlsx_files.append(os.path.join(_root, _file))

# Walk through the directory and its subdirectories
for _root, _dirs, _files in os.walk(_path_true):
    for _file in _files:
        if _file.endswith('.xlsx'):
            _xlsx_true_files.append(os.path.join(_root, _file))


for _file in _xlsx_files:
    _name = os.path.basename(os.path.dirname(_file))
    # Create export dir
    #os.mkdir(os.path.join(_output_dir, _name))
    for _file_true in _xlsx_true_files:
        if _name in _file_true:
            break

    # Copy all data
    shutil.copytree(os.path.dirname(_file_true), os.path.join(_output_dir, _name))
    
    # Override xlsx
    _df_changed = pd.read_excel(_file, index_col = 0)
    _df_original = pd.read_excel(_file_true, index_col = 0)
    _df_changed.loc["True_original"] = _df_original.loc["True", :]
    _df_changed.to_excel(os.path.join(_output_dir, _name, "results.xlsx")) 

### Calculate statitics

In [4]:
from Utils.evaluate_xlsx import *

In [7]:
_my_container = statisticContainer(r"cheatsheet_names.xlsx")
_my_container.load_dir(r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256")
_my_container.generate_statistics(true_key = "True")
_my_container.dump_data(f"FilteredData256")
_my_container.load_dir(r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256")
_my_container.generate_statistics(true_key = "True_original")
_my_container.dump_data(f"OrignialData256")

_my_container.export_to_xlsx("Filtered256Statistics.xlsx")


### Remap body parts 

In [ ]:
_remap_dict = {
    "skull": ["skull"],
    "shoulder": ["clavicula" ,"scapula"],
    "humerus": ["humerus"],
    "vertebrae_C": ["vertebrae_C"],
    "thorax": ["vertebrae_T", "sternum", "rib_1", "rib_2", "rib_3", "rib_4", "rib_5", "rib_6", "rib_7", "rib_8", "rib_9", "rib_10", "rib_11", "rib_12"],
    "vertebrae_L": ["vertebrae_L"],
    "forearm": ["ulna", "radius"],
    "pelvis": ["hip", "vertebrae_S", "sacrum"],
    "femur": ["femur"],
    "hand": ["carpal", "metacarpal", "phalanges_hand"],
    "patella": ["patella"],
    "shin": ["tibia", "fibula"],
    "tarsal": ["tarsal"],
    "foot":["metatarsal", "phalanges_feet"]
}


### Rules

And rule applies: (all bones must be present to give group label 1

* shoulder : clavicula & scapula

* forearm: ulna & radius

* pelvis: hip & vertebrae_S & sacrum

* hand: carpal & metacarpal & phalangs_hand

* foot: metatarsal & phalanges_feet

In [ ]:
def apply_rules(input_dict: dict = None)->dict:
    """
    Simple function which applies the rules defined above 

    Args:
        * input_dict, dict, dict generated by the cell below
    Output:
        * output_dict, dict, dict with 1 and 0 for each body part with
        rules abouve applied
    """
    # Create storage
    _output_dict = {}
    # Go through keys/body_parts in input dir and map them
    for _key in input_dict:
        # shoulder, forearm, foot
        if _key in ["shoulder", "forearm", "foot"]: 
            if input_dict[_key] == 2:
                _output_dict[_key] = 1
            else:
                _output_dict[_key] = 0
            continue
        # pelvis, hand
        if _key in ["pelvis", "hand"]: 
            if input_dict[_key] == 3:
                _output_dict[_key] = 1
            else:
                _output_dict[_key] = 0
            continue
        # EE
        if  input_dict[_key] >= 1:
            _output_dict[_key] = 1
        else:
            _output_dict[_key] = 0
    return _output_dict
        
        

#### Make remaped dir

#### Remaped with filtering

In [ ]:
_path = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256"
_output_dir = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256Remaped_withRules"

# List to store the paths of all .xlsx files
_xlsx_files = []

# Walk through the directory and its subdirectories
for _root, _dirs, _files in os.walk(_path):
    for _file in _files:
        if _file.endswith('.xlsx'):
            _xlsx_files.append(os.path.join(_root, _file))

for _file in _xlsx_files:
    _name = os.path.basename(os.path.dirname(_file))
    # Copy all data
    shutil.copytree(os.path.dirname(_file), os.path.join(_output_dir, _name))
    
    # Create new dict
    _output_dict = {}
    # Override xlsx
    _df = pd.read_excel(_file, index_col = 0)
    for _index in _df.index:
        # Skip area
        if _index == "Area":
            continue
        # Obtain row
        _row = _df.loc[_index]
        _row_dict = {}
        for _key in _remap_dict:
            _row_dict[_key] = 0
        # Fil data
        for _index_row in _row.index:
            if _row[_index_row] == 1:
                for _key in _remap_dict:
                    if _index_row.strip() in _remap_dict[_key]:
                        _row_dict[_key] += 1
        _output_dict[_index] = _row_dict
    for _key in list(_output_dict):
        _output_dict[_key] = apply_rules(_output_dict[_key])
    # Create _df & export
    _df = pd.DataFrame.from_dict(_output_dict, orient='index')
    _df.to_excel(os.path.join(_output_dir, _name, "results.xlsx"))


#### Remaped withouth filtering

In [ ]:
_path = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256"
_output_dir = r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256Remaped"

# List to store the paths of all .xlsx files
_xlsx_files = []

# Walk through the directory and its subdirectories
for _root, _dirs, _files in os.walk(_path):
    for _file in _files:
        if _file.endswith('.xlsx'):
            _xlsx_files.append(os.path.join(_root, _file))



for _file in _xlsx_files:
    _name = os.path.basename(os.path.dirname(_file))
    # Copy all data
    shutil.copytree(os.path.dirname(_file), os.path.join(_output_dir, _name))

    
    # Create new dict
    _output_dict = {}
    # Override xlsx
    _df = pd.read_excel(_file, index_col = 0)
    for _index in _df.index:
        # Skip area
        if _index == "Area":
            continue
        # Obtain row
        _row = _df.loc[_index]
        _row_dict = {}
        for _key in _remap_dict:
            _row_dict[_key] = 0
        # Fil data
        for _index_row in _row.index:
            if _row[_index_row] == 1:
                for _key in _remap_dict:
                    if _index_row.strip() in _remap_dict[_key]:
                        _row_dict[_key] = 1
                        
        _output_dict[_index] = _row_dict

    # Create _df & export
    _df = pd.DataFrame.from_dict(_output_dict, orient='index')
    _df.to_excel(os.path.join(_output_dir, _name, "results.xlsx"))
    

### Calculate statistics

In [5]:
_my_container = statisticContainer(r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\DataGenerationScripts\cheatsheet_names_remaped.xlsx")
_my_container.load_dir(r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256Remaped_withRules")
_my_container.generate_statistics(true_key = "True")
_my_container.dump_data(f"FilteredDatawRules")
_my_container.load_dir(r"C:\Users\CH258598\Desktop\FiteredDatasetNNMD256Subset\FilteredDataNNMD256Remaped_withRules")
_my_container.generate_statistics(true_key = "True_original")
_my_container.dump_data(f"OrignialDatawRules")

_my_container.export_to_xlsx("Filtered256StatisticsREMAPED_withRules.xlsx")
